### Hierarchical Indexing

* Higher-dimensional data : data indexed by more than one or two keys
* commonly referred to as multi-indexing
* In this section, we’ll explore the direct creation of MultiIndex objects; 
    * considerations around indexing, slicing, and computing statistics across multiply indexed data
    * useful routines for converting between simple and hierarchically indexed representations of your data.


In [105]:
import numpy as np
import pandas as pd

#### First Though ... "This Is Not The Way"

In [106]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), 
        ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]

In [107]:
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [108]:
# Can still access fairly straightforward
pop[('California', 2010): ('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [109]:
# However something like selecting all values from 2010 .. you have to get a bit messy
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

#### The better way: Pandas MultiIndex (.. This is the Way)

In [110]:
# Createa multi-index from the tuples as follows
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [111]:
index.levels

FrozenList([['California', 'New York', 'Texas'], [2000, 2010]])

In [112]:
# Reindex our series w/above MultiIndex
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

* In this multi-index representation, any blank entry indicates the same values as the line above it

In [113]:
# Access all data for which the  second index is 2010
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

#### MultiIndex as extra dimension
* You might notice something else here: we could easily have stored the same data using a simple DataFrame with index and column labels. 
* In fact, Pandas is built with this equivalence in mind. 
    * The unstack() method will quickly convert a multiply-indexed Series into a conventionally indexed DataFrame:

In [114]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [115]:
# Back to Series
pop_df.stack(), type(pop_df.stack())

(California  2000    33871648
             2010    37253956
 New York    2000    18976457
             2010    19378102
 Texas       2000    20851820
             2010    25145561
 dtype: int64,
 pandas.core.series.Series)

* What's the point?
    * Each extra level in a multi-index represents an extra dimension of data; taking advatanges of the flexibility

In [116]:
# Concretely, we might want to add another column of demographic data for each state at each year 
# (say, population under 18); with a MultiIndex this is as easy as adding another column to the DataFrame:

pop_df = pd.DataFrame({'total':pop, 'under18':[9267089, 9284094, 4687374, 4318033, 5906301,6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [117]:
frac_u18 = pop_df['under18'] - pop_df['total']
frac_u18

California  2000   -24604559
            2010   -27969862
New York    2000   -14289083
            2010   -15060069
Texas       2000   -14945519
            2010   -18266547
dtype: int64

In [118]:
frac_u18.unstack()

,2000,2010
California,-24604559,-27969862
New York,-14289083,-15060069
Texas,-14945519,-18266547


### Methods of MultiIndex Creation

In [119]:
df_1 = pd.DataFrame(np.random.rand(4,2), index=[['a', 'a', 'b', 'b'],[1,2,1,2]], 
                    columns=['data1', 'data2'])
df_1

data1     data2
a 1  0.392836  0.543110
  2  0.909790  0.181504
b 1  0.516117  0.168214
  2  0.848982  0.771179

In [120]:
data_mic = {('California', 2000): 33871648,('California', 2010): 37253956,('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561, ('New York', 2000): 18976457,('New York', 2010):19378102}
data_mic

{('California', 2000): 33871648,
 ('California', 2010): 37253956,
 ('Texas', 2000): 20851820,
 ('Texas', 2010): 25145561,
 ('New York', 2000): 18976457,
 ('New York', 2010): 19378102}

In [121]:
pd.Series(data_mic)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

#### Explicit MultiIndex constructors

In [122]:
# you can construct the MultiIndex from a simple list of arrays, giving the index values within each level:
pd_multi = pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1,2,1,2]])
pd_multi

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [123]:
pd_multi.levels, pd_multi.values

(FrozenList([['a', 'b'], [1, 2]]),
 array([('a', 1), ('a', 2), ('b', 1), ('b', 2)], dtype=object))

In [124]:
pd_multi[:2]

MultiIndex([('a', 1),
            ('a', 2)],
           )

In [125]:
pd_tups = pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

In [126]:
pd_tups.values

array([('a', 1), ('a', 2), ('b', 1), ('b', 2)], dtype=object)

* Similarly, you can construct the MultiIndex directly using its internal encoding by passing levels (a list of lists containing available index values for each level) and labels (a list of lists that reference these labels):


In [127]:
# MultiIndex level names
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [128]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for Columns

In [129]:
# MultiIndex for Columns

# hieararchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1,2]], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type']) 

# mock some data
data_muic = np.round(np.random.randn(4,6), 1)
data_muic[:, ::2] *= 10
data_muic += 37

# create the DataFrame
health_data = pd.DataFrame(data_muic, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  36.4  55.0  37.0  22.0  37.3
     2      54.0  36.6  47.0  34.4  48.0  38.4
2014 1      46.0  37.7  24.0  35.6  27.0  38.5
     2      32.0  38.4  42.0  37.5  32.0  34.3

* Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number. * With this in place we can, for example, index the top-level column by the person’s name and get a full DataFrame containing just that person’s information:

In [130]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      55.0  37.0
     2      47.0  34.4
2014 1      24.0  35.6
     2      42.0  37.5

In [131]:
health_data.index, health_data.columns

(MultiIndex([(2013, 1),
             (2013, 2),
             (2014, 1),
             (2014, 2)],
            names=['year', 'visit']),
 MultiIndex([(  'Bob',   'HR'),
             (  'Bob', 'Temp'),
             ('Guido',   'HR'),
             ('Guido', 'Temp'),
             (  'Sue',   'HR'),
             (  'Sue', 'Temp')],
            names=['subject', 'type']))

### Indexing and Slicing a MultiIndex

In [132]:
# A Series
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [133]:
# Single Elements by indexing w/multiple terms
pop['California', 2000]

33871648

In [134]:
# MultiIndex also supports partial indexing, or indexing just one of the levels in the index
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [135]:
pop.loc['California':'New York'] # inclusive recommend to use .loc for clarity but not required

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [136]:
# Partial Indexing on lower levels (just have to empty slice up until desired index level)
pop[:, 2010] # Just 2010 for each outer level state

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [137]:
# Selection Based on Boolean Masks
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [138]:
# Fancy Indexing
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [139]:
# Fancy Multiple Levels
pop.loc[[('California', 2000), ('Texas', 2010)]]

state       year
California  2000    33871648
Texas       2010    25145561
dtype: int64

In [140]:
## Back to Health Data
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  36.4  55.0  37.0  22.0  37.3
     2      54.0  36.6  47.0  34.4  48.0  38.4
2014 1      46.0  37.7  24.0  35.6  27.0  38.5
     2      32.0  38.4  42.0  37.5  32.0  34.3

In [141]:
health_data['Guido', 'HR'] # Guido's Heart Rate Data w/simple operation (subject then type) for multi-index column level

year  visit
2013  1        55.0
      2        47.0
2014  1        24.0
      2        42.0
Name: (Guido, HR), dtype: float64

In [142]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      34.0  36.4
     2      54.0  36.6

In [143]:
health_data.index, health_data.columns

(MultiIndex([(2013, 1),
             (2013, 2),
             (2014, 1),
             (2014, 2)],
            names=['year', 'visit']),
 MultiIndex([(  'Bob',   'HR'),
             (  'Bob', 'Temp'),
             ('Guido',   'HR'),
             ('Guido', 'Temp'),
             (  'Sue',   'HR'),
             (  'Sue', 'Temp')],
            names=['subject', 'type']))

In [144]:
# Each Individual index in loc or iloc can be passed a tuple of multiple indices
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        34.0
      2        54.0
2014  1        46.0
      2        32.0
Name: (Bob, HR), dtype: float64

In [145]:
# Count from 
health_data.iloc[:1, 3:5]

,subject,Guido,Sue
,type,Temp,HR
year,visit,,
2013,1,37.0,22.0


#### IndexSlice Object

In [146]:
# Review health_data
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  36.4  55.0  37.0  22.0  37.3
     2      54.0  36.6  47.0  34.4  48.0  38.4
2014 1      46.0  37.7  24.0  35.6  27.0  38.5
     2      32.0  38.4  42.0  37.5  32.0  34.3

In [147]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,34.0,55.0,22.0
2014,1,46.0,24.0,27.0


### Rearranging Multi-Indices
* Many of the MultiIndex slicing operations will fail if the index is not sorted (Example below)
    * Not "lexographically" sorted

In [148]:
idx = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data_ridx = pd.Series(np.random.rand(6), index=idx)
data_ridx.index.names = ['char', 'int']
data_ridx

char  int
a     1      0.223265
      2      0.348989
c     1      0.671973
      2      0.890028
b     1      0.010085
      2      0.477662
dtype: float64

In [149]:
try:
    data_ridx['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


* (Not the clearest message) but this is the result of the MultiIndex not being sorted
* For various reasons, partial slices and other similar operations require the levels in the MultiIndex to be in sorted (ie, lexographical) order
* sort_index(), and sortlevel() will be our friends here

In [150]:
data_ridx = data_ridx.sort_index()
data_ridx

char  int
a     1      0.223265
      2      0.348989
b     1      0.010085
      2      0.477662
c     1      0.671973
      2      0.890028
dtype: float64

In [151]:
data_ridx['a':'b']

char  int
a     1      0.223265
      2      0.348989
b     1      0.010085
      2      0.477662
dtype: float64

### Stacking and unstacking indices
* Common to push a hierarchical index to a column (this can be done by defining the level you'd like to transform (aka index level)

In [152]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [153]:
pop.unstack(level=0) # outer index (state level is pushed to column)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [154]:
pop.unstack(level=1) # nested index (year pushed to column)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


### Index setting & resetting
* Another way to rearrange hierarchical data is to turn the index labels into columns
    * This can be accomplished with the reset_index method
* Calling this will result in a DataFrame with your indexes push to a separate column (like state & year above)

In [160]:
print(type(pop))
display(pop)

<class 'pandas.core.series.Series'>


state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [162]:
# Series being transformed can have values named in the result DataFrame column
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [163]:
type(pop_flat)

pandas.core.frame.DataFrame

* set_index allows to build a MultiIndex from the column values
    * hierarchical set by level included in the call

In [164]:
# Original
pop_set_state_first = pop_flat.set_index(['state', 'year'])
pop_set_state_first

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [166]:
# Reset by year (looks a little silly and the state has multiple years so not the most useful but possible!)
pop_set_year_first = pop_flat.set_index(['year', 'state'])
pop_set_year_first

,,population
year,state,
2000,California,33871648
2010,California,37253956
2000,New York,18976457
2010,New York,19378102
2000,Texas,20851820
2010,Texas,25145561


### Data Aggregations on Multi-Indices
* For hierarchically indexed data, data agg methods (ex. mean(), min()) can be passed a level parameter
    * level controls which subset of the data the aggregate is computed on

In [167]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  36.4  55.0  37.0  22.0  37.3
     2      54.0  36.6  47.0  34.4  48.0  38.4
2014 1      46.0  37.7  24.0  35.6  27.0  38.5
     2      32.0  38.4  42.0  37.5  32.0  34.3

In [168]:
# Average measurements in the two visits each year (explore index level year)
hd_year_mean = health_data.mean(level='year')
hd_year_mean

/var/folders/r8/mmfd5rgj7v51_80zhnq8v9zc0000gn/T/ipykernel_26319/154423569.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  hd_year_mean = health_data.mean(level='year')


subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     44.0  36.50  51.0  35.70  35.0  37.85
2014     39.0  38.05  33.0  36.55  29.5  36.40

* Note here the textbook version has a soon to be deprecated way of accessing the level, see same result below for future usage

In [190]:
# Year average for each type of column measurement for subject (Guido = 51 is 2013's 55 and 2014's 47 avg)
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     44.0  36.50  51.0  35.70  35.0  37.85
2014     39.0  38.05  33.0  36.55  29.5  36.40

In [193]:
data_mean.mean(axis=1, level='type')

/var/folders/r8/mmfd5rgj7v51_80zhnq8v9zc0000gn/T/ipykernel_26319/1870386839.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean.mean(axis=1, level='type')


type,HR,Temp
year,,
2013,43.333333,36.683333
2014,33.833333,37.000000


In [222]:
# Return similar to above we can take the mean among levels 
# Specificy the level to groupby (we take the second level for columns, then perform the mean across the rows of data_mean)
type_agg_means = data_mean.groupby(level=[1], axis=1).mean()

type,HR,Temp
year,,
2013,43.333333,36.683333
2014,33.833333,37.000000


In [226]:
subj_agg_means_yearly = data_mean.groupby(level=[0], axis=1).mean()
subj_agg_means_yearly

subject,Bob,Guido,Sue
year,,,
2013,40.250,43.350,36.425
2014,38.525,34.775,32.950


In [227]:
# this will simply get mean for each rows and return the hierarchical column order for mean over years for type
data_mean.mean()

subject  type
Bob      HR      41.500
         Temp    37.275
Guido    HR      42.000
         Temp    36.125
Sue      HR      32.250
         Temp    37.125
dtype: float64